In [1]:
import pandas as pd 
import numpy as np
import torch

from torch import nn
import torch.nn.functional as F

In [2]:
x_train = pd.read_csv('../data/split/train/x_train.csv',index_col=0)
y_train = pd.read_csv('../data/split/train/y_train.csv',index_col=0)

x_test = pd.read_csv('../data/split/test/x_test.csv',index_col=0)
y_test = pd.read_csv('../data/split/test/y_test.csv',index_col=0)

In [4]:
x_train = torch.FloatTensor(x_train.to_numpy())
y_train = torch.FloatTensor(y_train.to_numpy())

x_test = torch.FloatTensor(x_test.to_numpy())
y_test = torch.FloatTensor(y_test.to_numpy())

In [29]:
train = torch.utils.data.TensorDataset(x_train,y_train)
test = torch.utils.data.TensorDataset(x_test,y_test)

In [30]:
train = torch.utils.data.DataLoader(train,batch_size=32)
test = torch.utils.data.DataLoader(test,batch_size=1)

In [7]:
next(iter(train))

[tensor([[ 1.0000,  0.0000, -0.2830,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -1.3225,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000,  1.0963,  ...,  0.0473,  0.8913,  0.0000],
         ...,
         [ 1.0000,  0.0000, -0.8203,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000,  0.4463,  ...,  0.0000,  0.0000,  0.0177],
         [ 0.0000,  0.0000, -0.4847,  ...,  0.0000,  0.9983,  0.0000]]),
 tensor([[4.0000],
         [3.5000],
         [4.2000],
         [3.8000],
         [4.2000],
         [3.0000],
         [3.8000],
         [3.0000],
         [4.4000],
         [4.4000],
         [4.0000],
         [3.6000],
         [3.7000],
         [4.4000],
         [4.0000],
         [4.0000],
         [3.2000],
         [2.8000],
         [3.6000],
         [2.6000],
         [3.9000],
         [3.6000],
         [3.4000],
         [4.4000],
         [3.5000],
         [3.8000],
         [2.9000],
         [3.0000],
         [3.8000],
       

In [8]:
x_train.shape

torch.Size([26738, 334])

In [10]:
model = nn.Sequential(
        nn.Linear((334),64),
        nn.ReLU(),
        nn.Linear(64,64),
        nn.ReLU(),
        nn.Linear(64,1)

)

In [11]:
from torchsummary import summary

In [12]:
summary(model)

Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            21,440
├─ReLU: 1-2                              --
├─Linear: 1-3                            4,160
├─ReLU: 1-4                              --
├─Linear: 1-5                            65
Total params: 25,665
Trainable params: 25,665
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            21,440
├─ReLU: 1-2                              --
├─Linear: 1-3                            4,160
├─ReLU: 1-4                              --
├─Linear: 1-5                            65
Total params: 25,665
Trainable params: 25,665
Non-trainable params: 0

In [18]:
x = x_train[:10]
y = y_train[:10]

In [19]:
yhat = model(x)
yhat

tensor([[ 0.0013],
        [ 0.0016],
        [-0.0064],
        [-0.0011],
        [-0.0066],
        [ 0.0093],
        [-0.0129],
        [ 0.0032],
        [-0.0329],
        [-0.0187]], grad_fn=<AddmmBackward0>)

In [20]:
lossfn = nn.MSELoss()

In [27]:
lossfn(yhat ,y).detach().numpy().item()

14.970410346984863

In [28]:
optim = torch.optim.Adam(model.parameters(),lr=0.01)

In [31]:
epochs = 10

train_loss = []
test_loss = []
for epoch in range(epochs):
    
    batch_loss = []
    for X,y in train:
  
        yhat = model(X)
        loss = lossfn(yhat ,y)
        
        batch_loss.append(loss.detach().numpy().item())
        
        optim.zero_grad()
        loss.backward()
        optim.step()
    
    with torch.no_grad():
        xx,yy= next(iter(test))
        
        yhat2 = model(xx)
        loss2 = lossfn(yhat2,yy)
        
    test_loss.append(loss2.detach().numpy().item())
    train_loss.append(np.mean(batch_loss))
    
    print(f' epoch: {epoch} train_loss: {np.mean(batch_loss)} test_loss: {loss2.detach().numpy().item()}')
    
        
        

 epoch: 0 train_loss: 0.21224163437449334 test_loss: 0.03513309732079506
 epoch: 1 train_loss: 0.09153618575096059 test_loss: 0.027294347062706947
 epoch: 2 train_loss: 0.0816997888132609 test_loss: 0.10184179246425629
 epoch: 3 train_loss: 0.07411555865218052 test_loss: 0.10806426405906677
 epoch: 4 train_loss: 0.0683791161332119 test_loss: 0.16917784512043
 epoch: 5 train_loss: 0.06330405619512335 test_loss: 0.18941687047481537
 epoch: 6 train_loss: 0.058904645128986625 test_loss: 0.1478276252746582
 epoch: 7 train_loss: 0.05481183327707734 test_loss: 0.1227341964840889
 epoch: 8 train_loss: 0.05033978323884415 test_loss: 0.15022577345371246
 epoch: 9 train_loss: 0.04462032263394463 test_loss: 0.1300368309020996


In [36]:
pred = model(x_test)
r2_score(y_test,pred.detach().numpy())

0.6697588502678133

In [34]:
from sklearn.metrics import r2_score

In [50]:
# Class

class make_model(nn.Module):
    
    def __init__(self,n1=32,n2=64,n_hidden=1):
        super().__init__()
        self.n1 = n1
        self.n2=n2
        self.hidden = n_hidden
        
        self.inp = nn.Linear(334,n1)
        self.hidden_layer = [nn.Linear(n1,n1) for _ 
                             in range(self.hidden)]
        
        self.fc1 = nn.Linear(n1,n2)
        self.out = nn.Linear(n2,1)
        
    def forward(self,X):
        
        x = F.relu(self.inp(X))
        for i in range(self.hidden):
            x = F.relu(self.hidden_layer[i](x))
        x = F.relu(self.fc1(x))
        x = self.out(x)
        
        return x

                
        

In [51]:
m = make_model()

In [52]:
m(x)

tensor([[-0.0879],
        [-0.0819],
        [-0.0869],
        [-0.0838],
        [-0.0890],
        [-0.0765],
        [-0.0827],
        [-0.0811],
        [-0.0862],
        [-0.0867]], grad_fn=<AddmmBackward0>)